## **Benchmarking Tool for Thin-Film Transistors** ⚡

## **Student**:

Rodrigo Santos Batista – rsb6@cin.ufpe.br

CIn ­ UFPE

## **Research project**:

Benchmarking emerging technologies for near and far computing

inside sensors (BEeTle)

## **Coordinator**:

Stefan Michael Blawid – sblawid@cin.ufpe.br

CIn ­ UFPE

## **Using the Model** 💻

Below, insert the address of the `MODULE_OTFT` module that contains the files to be imported. This folder contains all the necessary files that the model uses to function. It can be loaded following the example: `sys.path.append('/content/drive/MyDrive/your/path/Model_OTFT')`, simply replace the path below with the corresponding path from your mounted drive containing the module.

In [1]:
# Load the used classes
from modules_otft._imports import *
from modules_otft._read_data import ReadData
from modules_otft._model import TFTModel
from modules_otft._grafics import  TFTGraphicsPlot
from modules_otft._menu import TFTMenu

# from modules_otft._global_vars import *
from modules_otft._config import *
from modules_otft._utils import *
from modules_otft._display import display_settings

---
## **Instantiating the used classes**
---


In [2]:
plot = TFTGraphicsPlot()
menu = TFTMenu()
read = ReadData()

---
## **Loading sample data** 📚


---



In [3]:
# Read and show the path file Json
settings = enter_with_json_file()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


___________________________________________________________________________________ SETTINGS PRESENT IN THE JSON FILE:___________________________________________________________________________________

| path: /home/rsb6/Desktop/Artigo JICS/Model_OTFT/datas/SBMicro/tipo_p
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_transfer: A
-----------------------------------------------------------------------------------

In [4]:
ld_voltages = get_load_voltages(settings)

type_curve_plot = get_type_plot(settings)

shift_list = calculate_shift_list(settings)

list_tension_shift = get_shift_list(read, settings)

path_voltages = read.read_files_experimental(settings['path'], list_tension_shift)

Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(settings['type_read_data_exp'], path_voltages, settings['current_typic'], settings['experimental_data_scale_transfer'], settings['experimental_data_scale_output'], type_curve_plot)


In [5]:
display_settings(menu, settings, path_voltages, shift_list, list_tension_shift)


Settings:
---------------------------------
Type read data: read original data
Current typic : uA
Scale transfer: A
Scale output  : A
Shift value   : [0, -1, -2, -6]
List volt shift: [-40, -20, -39, -58, -74]
--------------------------------------------
--------------------------------------------
|                READ FILES                |
--------------------------------------------
--------------------------------------------
|             transfer-40V.csv             |
--------------------------------------------
|              output-20V.csv              |
--------------------------------------------
|              output-40V.csv              |
--------------------------------------------
|              output-60V.csv              |
--------------------------------------------
|              output-80V.csv              |
--------------------------------------------
--------------------------------------------

A tabela acima mostra todos os arquivos que foram lidos no diretório,

In [6]:
# Caso o Shift seja aplicado 
path_voltages, new_values_tension, new_list_tension = filter_and_load_files(read, settings, path_voltages, list_tension_shift)

Vv, Id, input_voltage, n_points, count_transfer, count_output = read.load_data(settings['type_read_data_exp'], path_voltages, settings['current_typic'], settings['experimental_data_scale_transfer'], settings['experimental_data_scale_output'], type_curve_plot)
list_tension_shift = new_values_tension
list_tension = new_list_tension


In [7]:
display_settings(menu, settings, path_voltages, shift_list, list_tension_shift)


Settings:
---------------------------------
Type read data: read original data
Current typic : uA
Scale transfer: A
Scale output  : A
Shift value   : [0, -1, -2, -6]
List volt shift: [-40, -20, -40, -60, -80]
--------------------------------------------
--------------------------------------------
|                READ FILES                |
--------------------------------------------
--------------------------------------------
|             transfer-40V.csv             |
--------------------------------------------
|              output-20V.csv              |
--------------------------------------------
|              output-40V.csv              |
--------------------------------------------
|              output-60V.csv              |
--------------------------------------------
|              output-80V.csv              |
--------------------------------------------
--------------------------------------------

A tabela acima mostra todos os arquivos que foram lidos no diretório,

In [8]:
tp_tst = get_transistor_type(settings)
resistance = get_resistance(settings)
current = get_current(settings)
width_t = get_transistor_width(settings)
load_parameters = load_coefficients(settings)


print()
menu.print_values(load_parameters)
print()

mode_idleak, load_idleak = load_idleak_parameters(settings)


   VTHO     |    DELTA   |      N     |     L      |    LAMBDA  |   VGCRIT   |    JTH     |     RS    |
-------------------------------------------------------------------------------------------------------
[1.0000e+01 │ 1.2300e-02 │ 1.0800e+02 │ 3.1800e+00 │ 1.4900e+03 │ 3.2300e+01 │ 1.0000e+00 │ 1.0000e+00]
-------------------------------------------------------------------------------------------------------



In [9]:
# @title ##### ***GRAFICS PLOT OPTION:*** { run: "auto" }
option = 'Show both curves' # @param ["Show transfer curve", "Show output curve", "Show both curves"]

plot = initialize_graphics_plot(settings)
model = instance_model(read, TFTModel, n_points, type_curve_plot, load_parameters, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, settings['experimental_data_scale_transfer'], settings['current_typic'], resistance, current)
in_model_data, in_exp_data, out_model_data, out_exp_data = load_experimental_data(read, count_transfer, Vv, Id, model, count_output)
plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data, out_model_data, in_exp_data, out_exp_data, shift_list, settings['select_files'], settings['current_typic'], type_curve_plot)
read.compute_relative_distance(Id, model)





**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 4301.1182

**************************************************************************************************************************************************


---
## **Model Optimization** ⚡
---

This optimization section focuses on estimating the optimal parameters for a MOSFET transistor given a sequence of input voltage $V$ [V] and current $I$ [mA], using the mathematical model introduced in section [5]. The goal here is to estimate these optimal parameters using the `curve_fit` function, as also introduced in section [4] (for more details, refer to that section). The parameters in question are listed below:

- $V_{tho}$: Gate threshold voltage of the transistor.
- $\Delta$: Channel modulation coefficient.
- $N$: Number of transistor channels.
- $L$: Channel length of the transistor.
- $\Lambda$: Channel modulation length.
- $V_{crit}$: Transistor breakdown voltage.
- $J_{th}$: Transistor saturation current.
- $R_s$: Transistor series resistance.



In [10]:
# @title #### **OPTIMIZING THE MODEL.** { run: "auto" }
model_id = create_model_opt(TFTModel, input_voltage, n_points, type_curve_plot, settings['current_typic'],
                            settings['experimental_data_scale_transfer'], load_idleak, mode_idleak, count_transfer,
                            resistance, current)

optimizer = create_optimizer(settings, path_voltages, type_curve_plot)

configure_optimizer(optimizer, settings)

coeff_opt, coeff_error, text_verbose = optimize_model(optimizer, model_id, load_parameters, *path_voltages)

# Exibição dos resultados
print('\n\n')
print('*' * 200)
print(text_verbose.capitalize())
print('*' * 200)
print('\n\n')


----------------------------------------------------------------------------------------------------
TRUST REGION REFLECTIVE (TRF) MODE
----------------------------------------------------------------------------------------------------




********************************************************************************************************************************************************************************************************
`xtol` termination condition is satisfied.
function evaluations 77, initial cost 7.6529e+05, final cost 1.4311e+04, first-order optimality 3.78e+00.

********************************************************************************************************************************************************************************************************





In [11]:
load_coefficient = load_coefficients(settings)

In [12]:
# @title #### **SHOW MODEL PARAMETERS OPTIMIZED.** { run: "auto" }
# @markdown ### **Loading Parameters and Optimizing the Model** ⚡
# @markdown > This section is dedicated to visualizing the parameters obtained through optimization. To do this, simply choose any of the options below:
# @markdown ##### ***Show the list coefficients optimized:***
# @markdown ##### **--> Vtho, delta, n, l, lam, Vgcrit, Jth, Rs**
# @markdown ---

option = 'Show the table of values opt' # @param ["Show the coeff values","Show the initial values", "Show the table of values opt", "coeff opt", "coeff error"]
show_model_parameters_optimized(menu, option, load_parameters, coeff_opt, coeff_error, current, settings['current_typic'], resistance)


---------------------------
 OPTIMIZED COEFFICIENTS:

╒══════════════════════╤═════════════════╤═══════════════════╤═════════════════════════════════════════╕
│ Parameter            │   Initial Value │   Optimized Value │   Associated Error (standard deviation) │
╞══════════════════════╪═════════════════╪═══════════════════╪═════════════════════════════════════════╡
│ VTHO [V]             │    10           │       8           │                             0.315301    │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ DELTA                │     0.0123      │       0.0108982   │                             0.00102892  │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤
│ N                    │   108           │      34.7733      │                            15.3074      │
├──────────────────────┼─────────────────┼───────────────────┼─────────────────────────────────────────┤


---
## **Comparing** ✅
---
This section aims to evaluate the model's performance relative to the initial data. Visual observation allows assessing whether the model significantly outperformed the initial coefficients.

In this section, simply execute the cells below; no explicit modifications are required.

In [13]:
# @title #### **Print grafics plots of model Optimizad and model data.** { run: "auto" }
# @markdown ### ***Choice the plot that you want to view:***
# @markdown ---
option = 'Show both curves comp' # @param ["Show Transfer curve opt", "Show output curve opt", "Show both curves opt", "Show Transfer curve comp", "Show output curve comp", "Show both curves comp"]

model_opt = create_optimized_model(read, TFTModel, n_points, type_curve_plot, coeff_opt, input_voltage, Vv, load_idleak, width_t, count_transfer, tp_tst, settings['experimental_data_scale_transfer'], settings['current_typic'], resistance, current)

in_model_data_opt, in_exp_data_opt, out_model_data_opt, out_exp_data_opt, in_model_data_comp, in_exp_data_comp, out_model_data_comp, out_exp_data_comp = get_model_data(read, count_transfer, count_output, Vv, Id, model_opt, model, compare=True)

plot_curves(option, plot, list_tension, list_tension_shift, count_transfer, in_model_data_comp, out_model_data_comp, in_exp_data_comp, out_exp_data_comp, shift_list, settings['select_files'], settings['current_typic'], type_curve_plot, compare=True)

read.compute_relative_distance(Id, model_opt)
print('\n\n')





**************************************************************************************************************************************************

|RELATIVE ERROR BETWEEN CURVES: 5100.6577

**************************************************************************************************************************************************





---
## **FineTuning Vtho**

In [14]:
vtho_calculation(in_model_data)